In [219]:
from PyDBDdim.utils import DimensionlessLearning
import torch
torch.manual_seed(0)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings("ignore")


In [220]:
#df = pd.ExcelFile(r"mass_balance_params.xlsx")
df_2 = pd.ExcelFile(r"data_from_EB_looping.xlsx")

In [221]:
# df_train_x['u_B_m_s'] = df_train_x['u_B_m_s'] / 1000 if np.min(df_train_x['u_B_m_s']) > 0 else df_train_x['u_B_m_s'] / 1000 - (np.min(df_train_x['u_B_m_s']) / 1000-1e-5)
# df_train_x['A_tot_m2'] = 1.
# df_train_x['t_a_s'] = df_train_x['t_a_s'] / 1e-7 if np.min(df_train_x['t_a_s']) > 0 else df_train_x['t_a_s'] / 1e-7 - (np.min(df_train_x['t_a_s']) / 1e-7-1e-5)
# df_train_x['t_b_s'] = df_train_x['t_b_s'] / 1e-4 if np.min(df_train_x['t_b_s']) > 0 else df_train_x['t_b_s'] / 1e-4 - (np.min(df_train_x['t_b_s']) / 1e-4-1e-5)
# df_train_x['Volume_m3'] = 1.
# df_train_x['K_iz_a_m3_s_atom'] = df_train_x['K_iz_a_m3_s_atom'] / 1e-15 if np.min(df_train_x['K_iz_a_m3_s_atom']) > 0 else df_train_x['K_iz_a_m3_s_atom'] / 1e-15 - (np.min(df_train_x['K_iz_a_m3_s_atom']) / 1e-15-1e-5)

# df_train_x['K_2_iz_a_m3_s_atom'] = df_train_x['K_2_iz_a_m3_s_atom'] / 1e-15 if np.min(df_train_x['K_2_iz_a_m3_s_atom']) > 0 else df_train_x['K_2_iz_a_m3_s_atom'] / 1e-15 - (np.min(df_train_x['K_2_iz_a_m3_s_atom']) / 1e-15-1e-5)

# df_train_x['K_iz_exc_a_m3_s_atom'] = df_train_x['K_iz_exc_a_m3_s_atom'] / 1e-14 if np.min(df_train_x['K_iz_exc_a_m3_s_atom']) > 0 else df_train_x['K_iz_exc_a_m3_s_atom'] / 1e-14 - (np.min(df_train_x['K_iz_exc_a_m3_s_atom']) / 1e-14-1e-5)

# df_train_x['n_sa_atoms_m3'] = df_train_x['n_sa_atoms_m3'] / 1e10 if np.min(df_train_x['n_sa_atoms_m3']) > 0 else df_train_x['n_sa_atoms_m3'] / 1e10 - (np.min(df_train_x['n_sa_atoms_m3']) / 1e10-1e-5)

# df_train_x['n_sb_atoms_m3'] = df_train_x['n_sb_atoms_m3'] / 1e10 if np.min(df_train_x['n_sb_atoms_m3']) > 0 else df_train_x['n_sb_atoms_m3'] / 1e10 - (np.min(df_train_x['n_sb_atoms_m3']) / 1e10-1e-5)

# df_train_x['A_a_m2'] = 1.

# df_train_x['A_b_m2'] = 1.

# df_train_x['n_He_exc_a_atoms_m3'] = df_train_x['n_He_exc_a_atoms_m3'] / 1e17 if np.min(df_train_x['n_He_exc_a_atoms_m3']) > 0 else df_train_x['n_He_exc_a_atoms_m3'] / 1e17 - (np.min(df_train_x['n_He_exc_a_atoms_m3']) / 1e17-1e-5)

def rescale(g):
    for i in range(0,g.shape[1]):
        if np.min(np.abs(g[:,i])) ==0:
            n = 0
        else:
            n = np.mean((np.log10(np.min(np.abs(g[:,i]))), np.log10(np.max(np.abs(g[:,i])))))
        if n<0:
                g[:,i] = g[:,i]/10**np.ceil(n) if np.min(g[:,i]) > 0 else g[:,i]/10**np.ceil(n) - (np.min(g[:,i]) / 10**np.ceil(n)-1e-5)
        else:
            g[:,i] = g[:,i]/10**np.floor(n) if np.min(g[:,i]) > 0 else g[:,i]/10**np.floor(n) - (np.min(g[:,i]) / 10**np.floor(n)-1e-5)
    return g

def rescale_vec(g):
    if np.min(np.abs(g)) ==0:
        n = 0
    else:
        n = np.mean((np.log10(np.min(np.abs(g))), np.log10(np.max(np.abs(g)))))
    if n<0:
            g= g/10**np.ceil(n) if np.min(g) > 0 else g/10**np.ceil(n) - (np.min(g) / 10**np.ceil(n)-1e-5)
    else:
        g = g/10**np.floor(n) if np.min(g) > 0 else g/10**np.floor(n) - (np.min(g) / 10**np.floor(n)-1e-5)
    return g

In [222]:
#                   0             1           2                    3                   4                   5           6                7                    8                     9                        10                  11                  12                      13                          
#df1_names = [ 't_a_seconds', 'Q_a_As', 'V_p_ta_kgm2_s3_A','T_e_a_kgm2_s3_A', 'n_He_exc_a_atoms_m3', 'u_B_a_m_s', 'v_e_a_m_s', 'K_2_iz_a_m3_s_atom','K_loss_a_m6_s_atom2', 'K_iz_exc_a_m3_s_atom', 'K_exc_a_m3_s_atom', 'K_iz_a_m3_s_atom', 'K_elastic_a_m3_s_atom', 'E_elastic_a_kgm2_s2' ]


df_time_a = rescale(np.array(pd.read_excel(df_2, sheet_name='time_a_data').iloc[:,1:]))
df_time_a_units = np.array(pd.read_excel(df_2, sheet_name='time_a_data_units').iloc[:,1:])
df_time_a_n = pd.read_excel(df_2, sheet_name='time_a_data_names').iloc[:,1:]
time_a_n= [df_time_a_n.iloc[0,i] for i in range(0,df_time_a_n.shape[1])]
#print(df_time_a_n)
#print(df_time_a_units)


#                   0             1                2               3                   4              5          6             7                   8                    9                       10              11                   12                   13                  
#df2_names =  [ 't_b_seconds', 'Q_b_As', 'V_p_tb_kgm2_s3_A','T_e_kgm2_s3_A', 'n_He_exc_atoms_m3', 'u_B_m_s', 'v_e_m_s','K_2_iz_m3_s_atom','K_loss_m6_s_atom2', 'K_iz_exc_m3_s_atom', 'K_exc_m3_s_atom', 'K_iz_m3_s_atom', 'K_elastic_m3_s_atom', 'E_elastic_kgm2_s2' ]
df_time_b = rescale(np.array(pd.read_excel(df_2, sheet_name='time_b_data').iloc[:,1:]))
df_time_b_units = np.array(pd.read_excel(df_2, sheet_name='time_b_data_units').iloc[:,1:])
df_time_b_n = pd.read_excel(df_2, sheet_name='time_b_data_names').iloc[:,1:]
time_b_n= [df_time_b_n.iloc[0,i] for i in range(0,df_time_b_n.shape[1])]


#                    0                   1                 2               3                 4               5               6                 7                  8           9        10        11                12                   13               14                   
#df3_names = ['E_period_kgm2_s2', 'n_sa_atoms_m3','n_sb_atoms_m3', 'n_e_electrons_m3', 'n_g_atoms_m3', 'T_g_kelvin', 'E_iz_kgm2_s2', 'E_iz_exc_kgm2_s2', 'E_exc_kgm2_s2', 'e_c_As', 'm_e_kg', 'M_He_kg',  'epsilon_A2s4_kg_m3', 'eps_0_A2s4_kg_m3', 'k_b_kgm2_s2_K']
df_other = rescale(np.array(pd.read_excel(df_2, sheet_name='other_data').iloc[:,1:]))
df_other_units = np.array(pd.read_excel(df_2, sheet_name='other_data_units').iloc[:,1:])
df_other_n = pd.read_excel(df_2, sheet_name='other_data_names').iloc[:,1:]
other_n= [df_other_n.iloc[0,i] for i in range(0,df_other_n.shape[1])]

#                    0                 1             2        3        4         5            6            7            8               9              10                 11                      12                      
#df4_names = ['Volume_rxtor_m2', 'V_all_beads_m2','A_a_m2','A_b_m2', 'h_m', 'Volume_m3', 'A_bead_m2', 'A_tot_m3', 'frequency_Hz', 'Flow_m3_s', 'temp_C_gas_K', 'Set_Voltage_kgm2_s3_A', 'pulse_time_seconds' ]
df_exp = rescale(np.array(pd.read_excel(df_2, sheet_name='Experiment_Design_data').iloc[:,1:]))
df_exp_units = np.array(pd.read_excel(df_2, sheet_name='Experiment_Data_units').iloc[:,1:])
df_exp_n = pd.read_excel(df_2, sheet_name='Experiment_Data_names').iloc[:,1:]
exp_n= [df_exp_n.iloc[0,i] for i in range(0,df_exp_n.shape[1])]



#base terms + F +K_iz, base terms: 'uB', 'Atot', 'ta', 'tb', 'Volume', 'Ka', 'K2a', 'Kexca', 'nsa', 'nsb',  'nHeexca,
#F is exp_n[9], K_iz is time_b_n[11]
#a = df_time_b[:,5].shape[0]
#inputs = np.hstack((df_time_b[:,5].reshape(a,1),df_exp[:,7].reshape(a,1),df_time_a[:,0].reshape(a,1),df_time_b[:,0].reshape(a,1),df_exp[:,5].reshape(a,1),df_time_a[:,11].reshape(a,1),df_time_a[:,7].reshape(a,1),df_time_a[:,9].reshape(a,1),df_other[:,1].reshape(a,1),df_exp[:,2].reshape(a,1),df_time_a[:,4].reshape(a,1),df_exp[:,9].reshape(a,1),df_time_b[:,11].reshape(a,1)))
#D_in = np.hstack((df_time_b_units[:,5].reshape(6,1),df_exp_units[:,7].reshape(6,1),df_time_a_units[:,0].reshape(6,1),df_time_b_units[:,0].reshape(6,1),df_exp_units[:,5].reshape(6,1),df_time_a_units[:,11].reshape(6,1),df_time_a_units[:,7].reshape(6,1),df_time_a_units[:,9].reshape(6,1),df_other_units[:,1].reshape(6,1),df_other_units[:,2].reshape(6,1),df_time_a_units[:,4].reshape(6,1),df_exp_units[:,9].reshape(6,1),df_time_b_units[:,11].reshape(6,1)))
#variables = [time_b_n[5],exp_n[7],time_a_n[0],time_b_n[0],exp_n[5],time_a_n[11],time_a_n[7],time_a_n[9],other_n[1],other_n[2],time_a_n[4],exp_n[9], time_b_n[11]]
#print(variables)
#print(D_in)


#base terms + F, base terms: 'uB', 'Atot', 'ta', 'tb', 'Volume', 'Ka', 'K2a', 'Kexca', 'nsa', 'nsb',  'nHeexca,
#F is exp_n[9]
#a = df_time_b[:,5].shape[0]
#inputs = np.hstack((df_time_b[:,5].reshape(a,1),df_exp[:,7].reshape(a,1),df_time_a[:,0].reshape(a,1),df_time_b[:,0].reshape(a,1),df_exp[:,5].reshape(a,1),df_time_a[:,11].reshape(a,1),df_time_a[:,7].reshape(a,1),df_time_a[:,9].reshape(a,1),df_other[:,1].reshape(a,1),df_exp[:,2].reshape(a,1),df_time_a[:,4].reshape(a,1),df_exp[:,9].reshape(a,1)))
#D_in = np.hstack((df_time_b_units[:,5].reshape(6,1),df_exp_units[:,7].reshape(6,1),df_time_a_units[:,0].reshape(6,1),df_time_b_units[:,0].reshape(6,1),df_exp_units[:,5].reshape(6,1),df_time_a_units[:,11].reshape(6,1),df_time_a_units[:,7].reshape(6,1),df_time_a_units[:,9].reshape(6,1),df_other_units[:,1].reshape(6,1),df_other_units[:,2].reshape(6,1),df_time_a_units[:,4].reshape(6,1),df_exp_units[:,9].reshape(6,1)))
#variables = [time_b_n[5],exp_n[7],time_a_n[0],time_b_n[0],exp_n[5],time_a_n[11],time_a_n[7],time_a_n[9],other_n[1],other_n[2],time_a_n[4],exp_n[9]]
#print(variables)
#print(D_in)


#base terms  'uB', 'Atot', 'ta', 'tb', 'Volume', 'Ka', 'K2a', 'Kexca', 'nsa', 'nsb',  'nHeexca,
a = df_time_b[:,5].shape[0]
inputs = np.hstack((df_time_b[:,5].reshape(a,1),df_exp[:,7].reshape(a,1),df_time_a[:,0].reshape(a,1),df_time_b[:,0].reshape(a,1),df_exp[:,5].reshape(a,1),df_time_a[:,11].reshape(a,1),df_time_a[:,7].reshape(a,1),df_time_a[:,9].reshape(a,1),df_other[:,1].reshape(a,1),df_exp[:,2].reshape(a,1),df_time_a[:,4].reshape(a,1)))
D_in = np.hstack((df_time_b_units[:,5].reshape(6,1),df_exp_units[:,7].reshape(6,1),df_time_a_units[:,0].reshape(6,1),df_time_b_units[:,0].reshape(6,1),df_exp_units[:,5].reshape(6,1),df_time_a_units[:,11].reshape(6,1),df_time_a_units[:,7].reshape(6,1),df_time_a_units[:,9].reshape(6,1),df_other_units[:,1].reshape(6,1),df_other_units[:,2].reshape(6,1),df_time_a_units[:,4].reshape(6,1)))
variables = [time_b_n[5],exp_n[7],time_a_n[0],time_b_n[0],exp_n[5],time_a_n[11],time_a_n[7],time_a_n[9],other_n[1],other_n[2],time_a_n[4]]
print(variables)
print(D_in)

#base terms -A,V 'uB',  'ta', 'tb', 'Volume', 'Ka', 'K2a', 'Kexca', 'nsa', 'nsb',  'nHeexca,
# a = df_time_b[:,5].shape[0]
# inputs = np.hstack((df_time_b[:,5].reshape(a,1),df_time_a[:,0].reshape(a,1),df_time_b[:,0].reshape(a,1),df_time_a[:,11].reshape(a,1),df_time_a[:,7].reshape(a,1),df_time_a[:,9].reshape(a,1),df_other[:,1].reshape(a,1),df_exp[:,2].reshape(a,1),df_time_a[:,4].reshape(a,1)))
# D_in = np.hstack((df_time_b_units[:,5].reshape(6,1),df_time_a_units[:,0].reshape(6,1),df_time_b_units[:,0].reshape(6,1),df_time_a_units[:,11].reshape(6,1),df_time_a_units[:,7].reshape(6,1),df_time_a_units[:,9].reshape(6,1),df_other_units[:,1].reshape(6,1),df_other_units[:,2].reshape(6,1),df_time_a_units[:,4].reshape(6,1)))
# variables = [time_b_n[5],time_a_n[0],time_b_n[0],time_a_n[11],time_a_n[7],time_a_n[9],other_n[1],other_n[2],time_a_n[4]]
# print(variables)
# print(D_in)

#to compare to ketong's original code only
#not to construct  'uB', 'Atot', 'ta', 'tb', 'Volume', 'Ka', 'K2a', 'Kexca', 'nsa', 'nsb', 'Aa', 'Ab', 'nHeexca,
# a = df_time_b[:,5].shape[0] #this is just the number of data points
# inputs = np.hstack((df_time_b[:,5].reshape(a,1),df_exp[:,7].reshape(a,1),df_time_a[:,0].reshape(a,1),df_time_b[:,0].reshape(a,1),df_exp[:,5].reshape(a,1),df_time_a[:,11].reshape(a,1),df_time_a[:,7].reshape(a,1),df_time_a[:,9].reshape(a,1),df_other[:,1].reshape(a,1),df_other[:,2].reshape(a,1),df_exp[:,2].reshape(a,1),df_exp[:,3].reshape(a,1),df_time_a[:,4].reshape(a,1)))
# D_in = np.hstack((df_time_b_units[:,5].reshape(6,1),df_exp_units[:,7].reshape(6,1),df_time_a_units[:,0].reshape(6,1),df_time_b_units[:,0].reshape(6,1),df_exp_units[:,5].reshape(6,1),df_time_a_units[:,11].reshape(6,1),df_time_a_units[:,7].reshape(6,1),df_time_a_units[:,9].reshape(6,1),df_other_units[:,1].reshape(6,1),df_other_units[:,2].reshape(6,1),df_exp_units[:,2].reshape(6,1),df_exp_units[:,3].reshape(6,1),df_time_a_units[:,4].reshape(6,1)))
# variables = [time_b_n[5],exp_n[7],time_a_n[0],time_b_n[0],exp_n[5],time_a_n[11],time_a_n[7],time_a_n[9],other_n[1],other_n[2],exp_n[2],exp_n[3],time_a_n[4]]
# print(variables)
# print(D_in)


['u_B_m_s', 'A_tot_m3', 't_a_seconds', 't_b_seconds', 'Volume_m3', 'K_iz_a_m3_s_atom', 'K_2_iz_a_m3_s_atom', 'K_iz_exc_a_m3_s_atom', 'n_sa_atoms_m3', 'n_sb_atoms_m3', 'n_He_exc_a_atoms_m3']
[[ 1  2  0  0  3  3  3  3 -3 -3 -3]
 [-1  0  1  1  0 -1 -1 -1  0  0  0]
 [ 0  0  0  0  0 -1 -1 -1  1  1  1]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]]


In [223]:
print(inputs.shape)
print(D_in.shape)

(166, 11)
(6, 11)


In [224]:
print(inputs[:,10])

[1.26113486 3.10828496 2.54307505 2.05619627 1.20402122 1.2993149
 1.30526692 1.15823665 1.34387547 1.42088813 1.60032635 1.33582501
 1.48405186 1.48523378 1.44093946 1.4505196  1.60338152 1.98006436
 1.9619605  1.78849676 1.83543198 2.01147344 1.91580601 1.98361517
 1.91017132 1.72902634 2.20856355 1.97487417 1.90063144 1.76579201
 1.80436684 1.82210651 1.83056756 2.27180795 2.1612181  2.18948516
 2.16765091 1.897429   1.81537044 1.77567056 2.42912499 1.65745035
 2.30081106 2.64058556 2.62069997 2.47770082 2.43801579 2.61674862
 3.01682255 2.98551177 2.42120857 2.5121597  1.45937712 2.41146737
 2.69539813 3.5406239  4.76007049 3.61957343 3.51873218 3.25990083
 3.90437076 3.24477505 3.27964645 6.95023274 3.5316073  3.89892172
 3.9778323  5.33198036 0.67153854 0.66277982 0.56847795 1.06035583
 0.69904743 0.91830486 0.7752149  1.03536608 1.23430341 1.096202
 1.13535006 1.11327591 1.13980388 1.03550709 1.02522166 0.48361467
 0.63737485 0.61924916 0.55005514 0.89612288 1.03649535 1.0361509

In [225]:
#For predicting Te/Tg: keep top block, comment bottom block. For predicting ne/ng, keep bottom block, comment top block.




#### TOP BLOCK ####

# df_out = pd.read_excel(df_2, sheet_name='time_b_data').iloc[:,1:]
# T_e = np.array(df_out.iloc[:,3])
# df_out = pd.read_excel(df_2, sheet_name='other_data').iloc[:,1:]
# T_g = np.array(df_out.iloc[:,5])*0.026/297
# T_e_no_dim = T_e/T_g
# print(T_e)
# output = rescale_vec(T_e_no_dim)
# print(output)
# D_out = np.array(
#     [
#         [0.],
#         [0.],
#         [0.],
#     ]
# )




#### BOTTOM BLOCK ####

df_out = pd.read_excel(df_2, sheet_name='other_data').iloc[:,1:]
n_e = np.array(df_out.iloc[:,3])
df_out = pd.read_excel(df_2, sheet_name='other_data').iloc[:,1:]
n_g = np.array(df_out.iloc[:,4])
n_e_no_dim = n_e/n_g
print(n_e)
print(n_e_no_dim)
output = rescale_vec(n_e_no_dim)
print(output)
D_out = np.array(
    [
        [0.],
        [0.],
        [0.],
        [0.],
        [0,],
        [0.]
    ],

)


[1.58309060e+12 4.09791587e+12 3.70354896e+12 3.09336680e+12
 1.91952999e+12 2.21022048e+12 2.23385412e+12 2.13639165e+12
 2.35983478e+12 2.49036030e+12 3.03167523e+12 2.84210352e+12
 3.18668511e+12 3.44177523e+12 3.54603983e+12 3.67296116e+12
 4.31186248e+12 4.33912928e+12 4.73376403e+12 5.00551030e+12
 4.98515118e+12 5.72909992e+12 5.89117393e+12 6.26848555e+12
 6.45399992e+12 6.20983876e+12 5.67551914e+12 5.45777510e+12
 5.72671834e+12 5.61130951e+12 6.00917947e+12 6.64918696e+12
 6.84402475e+12 8.57360434e+12 6.40318992e+12 6.75711285e+12
 7.31469123e+12 6.75996292e+12 7.03896384e+12 7.29018020e+12
 1.03134954e+13 9.16691521e+12 1.08864078e+13 8.43394263e+12
 9.14620112e+12 9.22868952e+12 1.08388820e+13 1.36449201e+13
 1.13943161e+13 1.32970690e+13 1.25674808e+13 1.53250521e+13
 2.04441246e+12 4.12911430e+12 5.94276348e+12 9.54453782e+12
 1.54484782e+13 1.35689855e+13 1.51597471e+13 1.11442229e+13
 1.61075216e+13 1.58773965e+13 3.97798005e+12 1.06795461e+13
 8.02225240e+12 1.212793

In [226]:
fff = DimensionlessLearning(D_in, D_out)
fff.basis_col

tensor([[ 1.0000,  1.0000,  0.0000,  0.0000,  0.0000, -1.0000, -1.0000, -1.0000],
        [-0.5000, -0.5000, -1.5000,  0.0000,  0.0000,  0.5000,  0.5000,  0.5000],
        [ 1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000, -1.0000, -1.0000,  1.0000,  1.0000,  1.0000],
        [ 0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000]],
       dtype=torch.float64)

In [227]:
fff.basis_col

tensor([[ 1.0000,  1.0000,  0.0000,  0.0000,  0.0000, -1.0000, -1.0000, -1.0000],
        [-0.5000, -0.5000, -1.5000,  0.0000,  0.0000,  0.5000,  0.5000,  0.5000],
        [ 1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000, -1.0000, -1.0000,  1.0000,  1.0000,  1.0000],
        [ 0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000]],
       dtype=torch.float64)

In [228]:
fff.read_data(inputs, output)


In [229]:
from PyDBDdim import PiLinearRegressionViaTorch


#the code does five fold to ensure the R2 is worth while.
# we then want to rerun the fitting on all data instead of 80% of it. The 5fold find the right number of epochs.
# so, we re run to find the right number of epochs, and then we must store the parameters and the R2 from the 5-fold fitting.
#

In [230]:
#UPDATE RUN PARAMETERS. POLYNOMIAL ORDER? # of dim groups? Which variables are called? etc.


ndimensionless = 1

lambda_gamma = 0.0003 #lambda_gamma = 0.01, #replaced
seed = 22

poly_order = 1

poly_mapping = np.array([[0],
                         [1]])

#poly_mapping = np.array([[0, 0],
#                         [1, 0],
#                         [0, 1],
#                         [2, 0],
#                         [1, 1],
#                         [0, 2]])
lambda_beta = 0.01 #maybe cut in half idk
w_array = np.array(fff._basis_col)  #this is the w array, that is, the columns in Null(D)
gamma_name = ['y'+str(id) for id in range(0,fff.basis_col.shape[1]) ]
beta_name = ['b'+str(id) for id in range(0,poly_mapping.shape[0]) ]
poly_name = ['dim'+str(id+1) for id in range(0,ndimensionless)]



metric = 'r2'
para_threshold = 0.005
beta_threshold = 0.005
training_epochs =10000
score = []

import numpy as np
from sklearn.model_selection import train_test_split

#create test set
X_train_val, X_test, y_train_val, y_test = train_test_split(fff.X, fff.y, test_size=0.2, random_state=42)

# create validation and training set
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

####################################################
# SOLVE FOR DIMENSIONLESS NUMBERS



In [231]:
#set save path

current_path = r"C:\Users\vvmil\Documents\Python_Vmil\Jupyter_Notebooks\Plasma_He_calcs\dimensionless_numbers_mb\limited_terms\single_lambdas"
#new_folder =r'mb_1dim_v1_pt0001_pt_01_order1_golden_child'
new_folder = str(ndimensionless)+r'dim_order'+str(poly_order)+r'_lambda'+str(lambda_gamma)+r"_seed"+str(seed)
new_path = current_path+'\\'+new_folder
if os.path.exists(new_path):
    print(f"File '{new_path}' already exists.")
else:
    os.mkdir(new_path)       
    print(f"File '{new_path}' did not exist.")

file_path = new_path+'\\'+new_folder +'_seed'+str(seed)+ '.xlsx'
fig_path_dim1 = new_path+'\\'+new_folder+"dim1"
fig_path_dim2 = new_path+'\\'+new_folder+"dim2"
print(fig_path_dim1)

File 'C:\Users\vvmil\Documents\Python_Vmil\Jupyter_Notebooks\Plasma_He_calcs\dimensionless_numbers_mb\limited_terms\single_lambdas\1dim_order1_lambda0.0003_seed22' did not exist.
C:\Users\vvmil\Documents\Python_Vmil\Jupyter_Notebooks\Plasma_He_calcs\dimensionless_numbers_mb\limited_terms\single_lambdas\1dim_order1_lambda0.0003_seed22\1dim_order1_lambda0.0003_seed22dim1


In [232]:


#find which hyperparameter performed best

# idx = np.arange(lg_set.shape[0])
# best_hp = idx[array_metric_num[:,0]==np.max(array_metric_num)]

# if len(best_hp.shape)>1:
#     print('the following are equivalent ',best_hp)
#     print(best_hp.shape)
#     best_hp = best_hp[0,1]


#22 best so far, R2 = 0.998 for lambda = 0.001
#14 best so far, R2 = 0.945
torch.manual_seed(seed)

X_train_val, X_test, y_train_val, y_test = train_test_split(fff.X, fff.y, test_size=0.2, random_state=42)



best_hp = lambda_gamma


model_train = PiLinearRegressionViaTorch.TrainHolder(X_train_val, y_train_val, poly_mapping.shape[0], fff.y.shape[-1], poly_mapping, fff.basis_col, ndimensionless, lambda_gamma, lambda_beta, lowest_para_threshold=para_threshold)
metric_num = model_train.train(training_epochs, True, val_x=X_test, val_y =y_test , metric=metric, norm_on='null_space')

model_train.model.load_state_dict(torch.load('best_model.pt'))


r2 = model_train.get_validation_metric(X_test,y_test,metric)
print(r2)

paras = [j for j in model_train.model.parameters()]
gamma = paras[0].detach()
beta = paras[1].detach()

#clamp them
t = gamma*(torch.abs(gamma) > para_threshold)
beta_prune = beta*(torch.abs(beta) > beta_threshold)


#store values from training
best_metric_num = np.array(metric_num)
best_beta = beta_prune

best_dim1_gamma = np.asarray(t.transpose(0,1)[0]) # gamma corresponding to dimensionless number 1
best_cardinality_dim1_gamma = np.sum(np.where(best_dim1_gamma==0,0,1)) #non-zero elements corresponding to gamma for dim # 1
best_dim1_w = np.matmul(w_array, best_dim1_gamma)  # w corresponding to dimensionless number 1
best_cardinality_dim1_w = np.sum(np.where(best_dim1_w==0,0,1)) #non-zero elements in w corresponding to dim #1

print(t)
print(np.asarray(t.transpose(0,1)[0]))
print(best_dim1_gamma)



if ndimensionless >=2:
    best_dim2_gamma = np.asarray(t.transpose(0,1)[1]) # gamma corresponding to dimensionless number 2
    best_cardinality_dim2_gamma = np.sum(np.where(best_dim2_gamma==0,0,1)) #non-zero elements corresponding to gamma for dim # 2
    best_dim2_w = np.matmul(w_array, best_dim2_gamma)  # w corresponding to dimensionless number 2
    best_cardinality_dim2_w = np.sum(np.where(best_dim2_w==0,0,1)) #non-zero elements in w corresponding to dim #2


#r2, lambda, beta vector , cardinality 
best_data_dim1 = np.hstack((best_metric_num.reshape(1,1),   np.array([best_hp]).reshape(1,1),  best_beta[0].reshape(1,poly_mapping.shape[0]),  best_cardinality_dim1_gamma.reshape(1,1) ,best_dim1_gamma.reshape(1,best_dim1_gamma.shape[0]) ,      best_dim1_w.reshape(1,best_dim1_w.shape[0]),   best_cardinality_dim1_w.reshape(1,1)))
data_name =  [ metric] + ['lambda']+ [beta_name[i] for i in range(0,len(beta_name))]+['cardinality_of_gamma_dim1']+[gamma_name[i] for i in range(0,len(gamma_name))]+[variables[i] for i in range(0,len(variables))]+['cardinality_of_w_dim1']

df_best_dim1 = pd.DataFrame(data=best_data_dim1, columns = [ metric] + ['lambda']+ [beta_name[i] for i in range(0,len(beta_name))]+['cardinality_of_gamma_dim1']+[gamma_name[i] for i in range(0,len(gamma_name))]+[variables[i] for i in range(0,len(variables))]+['cardinality_of_w_dim1'])

##### BELOW MUST BE CORRECTED#####
if ndimensionless >=2:
    
    best_data_dim2 = np.hstack((best_metric_num.reshape(1,1),   best_hp.reshape(1,1),  best_beta[0].reshape(1,poly_mapping.shape[0]),   best_cardinality_dim2_gamma.reshape(1,1),    best_dim2_w.reshape(1,best_dim1_w.shape[0]),   best_cardinality_dim2_gamma.reshape(1,1)))
    df_best_dim2 = pd.DataFrame(data=best_data_dim2, columns = [ metric] + ['lambda']+ [beta_name[i] for i in range(0,len(beta_name))]+['cardinality_of_gamma_dim1']+[variables[i] for i in range(0,len(variables))]+['cardinality_of_w_dim1'])

#df_best_dim1 = pd.DataFrame(data=D_in,columns=variables) #first sheet, list dimension vector
#####

loss: 0.0009089191133882072 at iteration 9999, val_loss: 0.996077240363745, best_val_loss: 0.9981022642558471: 100%|██████████| 10000/10000 [00:33<00:00, 297.93it/s]


0.9981022642558471
tensor([[-0.0000],
        [-0.1764],
        [-0.4219],
        [ 0.0000],
        [ 0.4845],
        [-0.1064],
        [-0.8641],
        [ 0.9589]], dtype=torch.float64)
[-0.         -0.17638469 -0.42190232  0.          0.48448349 -0.10637623
 -0.86407044  0.95887587]
[-0.         -0.17638469 -0.42190232  0.          0.48448349 -0.10637623
 -0.86407044  0.95887587]


In [233]:
#best_data_dim1 = np.hstack((best_metric_num.reshape(1,1),   np.array([best_hp]).reshape(1,1),  best_beta[0].reshape(1,poly_mapping.shape[0]),   best_cardinality_dim1_gamma.reshape(1,1),    best_dim1_w.reshape(1,best_dim1_w.shape[0]),   best_cardinality_dim1_gamma.reshape(1,1)))
print(best_metric_num.reshape(1,1).shape)
print(np.array([best_hp]).reshape(1,1).shape)
print(best_beta[0].reshape(1,poly_mapping.shape[0]).shape)
print(best_cardinality_dim1_gamma.reshape(1,1).shape)
print(best_dim1_w.reshape(1,best_dim1_w.shape[0]).shape)
print(best_cardinality_dim1_gamma.reshape(1,1).shape)

(1, 1)
(1, 1)
torch.Size([1, 2])
(1, 1)
(1, 11)
(1, 1)


In [234]:
#best_data_dim1 = np.hstack((best_metric_num.reshape(1,1),   np.array([best_hp]).reshape(1,1),  best_beta[0].reshape(1,poly_mapping.shape[0]),  best_cardinality_dim1_gamma.reshape(1,1) ,best_dim1_gamma.reshape(1,best_dim1_gamma.shape[0]) ,    best_cardinality_dim1_gamma.reshape(1,1),    best_dim1_w.reshape(1,best_dim1_w.shape[0]),   best_cardinality_dim1_gamma.reshape(1,1)))
#data_name =  [ metric] + ['lambda']+ [beta_name[i] for i in range(0,len(beta_name))]+['cardinality_of_gamma_dim1']+[gamma_name[i] for i in range(0,len(gamma_name))]+[variables[i] for i in range(0,len(variables))]+['cardinality_of_w_dim1']
#print(best_metric_num.reshape(1,1).shape)
#print(np.array([best_hp]).reshape(1,1).shape)
#print(best_beta[0].reshape(1,poly_mapping.shape[0]).shape)
#print(best_cardinality_dim1_gamma.reshape(1,1).shape)
#print(best_dim1_gamma.reshape(1,best_dim1_gamma.shape[0]).shape)
print(best_cardinality_dim1_gamma.reshape(1,1).shape)
print(best_dim1_w.reshape(1,best_dim1_w.shape[0]).shape)
print(best_cardinality_dim1_gamma.reshape(1,1).shape)

(1, 1)
(1, 11)
(1, 1)


In [235]:
#[ metric] + ['lambda']+ [beta_name[i] for i in range(0,len(beta_name))]+['cardinality_of_gamma_dim1']+[gamma_name[i] for i in range(0,len(gamma_name))]+[variables[i] for i in range(0,len(variables))]+['cardinality_of_w_dim1'])print(len([ metric]))
#print(len([metric]))
#print(len( ['lambda']))
#print(len( [beta_name[i] for i in range(0,len(beta_name))]))
#print(len(['cardinality_of_gamma_dim1']))
#print(len([gamma_name[i] for i in range(0,len(gamma_name))]))
print(len([variables[i] for i in range(0,len(variables))]))
print(len(['cardinality_of_w_dim1']))

11
1


In [236]:
print(best_cardinality_dim1_w)
print(best_dim1_w)
#best_cardinality_dim1_w = np.sum(np.where(best_dim1_w==0,0,1))

9
[-0.1648139   0.71526042  0.         -0.17638469 -0.42190232 -0.49605429
  0.          0.48448349 -0.10637623 -0.86407044  0.95887587]


In [237]:
model_train.model.load_state_dict(torch.load('best_model.pt'))
print([i for i in model_train.model.parameters()])

[Parameter containing:
tensor([[-7.0663e-04],
        [-1.7638e-01],
        [-4.2190e-01],
        [ 1.0740e-04],
        [ 4.8448e-01],
        [-1.0638e-01],
        [-8.6407e-01],
        [ 9.5888e-01]], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([[-0.0048,  0.0121]], dtype=torch.float64, requires_grad=True)]


In [238]:
paras = [j for j in model_train.model.parameters()]
gamma = paras[0].detach()
beta = paras[1].detach()

#clamp them
t = gamma*(torch.abs(gamma) > para_threshold)
beta_prune = beta*(torch.abs(beta) > beta_threshold)
print(t)
print(np.asarray(t.transpose(0,1)[0]))
print(best_dim1_gamma)

tensor([[-0.0000],
        [-0.1764],
        [-0.4219],
        [ 0.0000],
        [ 0.4845],
        [-0.1064],
        [-0.8641],
        [ 0.9589]], dtype=torch.float64)
[-0.         -0.17638469 -0.42190232  0.          0.48448349 -0.10637623
 -0.86407044  0.95887587]
[-0.         -0.17638469 -0.42190232  0.          0.48448349 -0.10637623
 -0.86407044  0.95887587]


In [239]:
# if you need the orignial pis
readable_para = torch.cat((t.transpose(0,1).reshape(-1), torch.tensor([-8, poly_order])))
ori_pis = fff.get_symbolic_pis(readable_para, variables)
ori_pis[0]
#ori_pis[1]


A_tot_m3**0.71526042488953*K_iz_exc_a_m3_s_atom**0.484483488098862*n_He_exc_a_atoms_m3**0.958875872823819/(K_iz_a_m3_s_atom**0.496054285912162*Volume_m3**0.421902317793513*n_sa_atoms_m3**0.106376230038761*n_sb_atoms_m3**0.864070440598358*t_b_seconds**0.176384694211821*u_B_m_s**0.164813896398521)

In [240]:
ori_pis[0]
print(t)
print(np.asarray(t.transpose(0,1)[0]))
print(best_dim1_gamma)

tensor([[-0.0000],
        [-0.1764],
        [-0.4219],
        [ 0.0000],
        [ 0.4845],
        [-0.1064],
        [-0.8641],
        [ 0.9589]], dtype=torch.float64)
[-0.         -0.17638469 -0.42190232  0.          0.48448349 -0.10637623
 -0.86407044  0.95887587]
[-0.         -0.17638469 -0.42190232  0.          0.48448349 -0.10637623
 -0.86407044  0.95887587]


In [241]:
print(np.matmul(D_in,best_dim1_w))

[2.22044605e-16 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00]


In [242]:
# File path of the Excel file

if os.path.exists(file_path):
    # Delete the file
    os.remove(file_path)
    print(f"File '{file_path}' deleted successfully.")
else:
    print(f"File '{file_path}' does not exist.")


df0 = pd.DataFrame(data=D_in,columns=variables) #first sheet, list dimension vector

df1 = pd.DataFrame(data=np.transpose(w_array),columns=variables) #second sheet, list nullspace vectors and associated variables


df4 =  pd.DataFrame(data = best_beta, columns =  [beta_name[i] for i in range(0,len(beta_name))] )
df5 = pd.DataFrame(data = poly_mapping, columns = [poly_name[i] for i in range(0,ndimensionless)])
with pd.ExcelWriter(file_path) as writer:  
    df0.to_excel(writer, sheet_name='dimension_matrix')
    df1.to_excel(writer, sheet_name='null_space_matrix')
    df4.to_excel(writer, sheet_name='beta')
    df5.to_excel(writer, sheet_name='polynomial')
    df_best_dim1.to_excel(writer, sheet_name='best_dim_1')

    
    
print(t)
print(np.asarray(t.transpose(0,1)[0]))
print(best_dim1_gamma)

File 'C:\Users\vvmil\Documents\Python_Vmil\Jupyter_Notebooks\Plasma_He_calcs\dimensionless_numbers_mb\limited_terms\single_lambdas\1dim_order1_lambda0.0003_seed22\1dim_order1_lambda0.0003_seed22_seed22.xlsx' does not exist.
tensor([[-0.0000],
        [-0.1764],
        [-0.4219],
        [ 0.0000],
        [ 0.4845],
        [-0.1064],
        [-0.8641],
        [ 0.9589]], dtype=torch.float64)
[-0.         -0.17638469 -0.42190232  0.          0.48448349 -0.10637623
 -0.86407044  0.95887587]
[-0.         -0.17638469 -0.42190232  0.          0.48448349 -0.10637623
 -0.86407044  0.95887587]


In [243]:
#import winsound
#frequency = 2500  # Set Frequency To 2500 Hertz
#duration = 150  # Set Duration To 1000 ms == 1 second
#winsound.Beep(frequency, duration)

In [244]:
len(beta_name)

2